In [1]:
import torch
import numpy as np
import pandas as pd
import math
import dgl
import pickle
from multisage.builder import PandasGraphBuilder
import ast

In [2]:
df = pd.read_excel('dataset/Beauty/data.xlsx')

In [3]:
for i in range(len(df)):
    df['특징'][i] = ast.literal_eval(df['특징'][i])
    new_features = []
    for j in df['특징'][i]:
        if ', ' in j:
            new_features.extend(j.split(', '))
            for k in j.split(', '):
                if ',' in k:
                    new_features.extend(k.split(','))
        elif ',' in j:
            new_features.extend(j.split(','))
            for k in j.split(','):
                if ', ' in k:
                    new_features.extend(k.split(', '))
        else:
            new_features.append(j)
    df.at[i, '특징'] = new_features

for i in range(len(df)):
    df['느낌'][i] = ast.literal_eval(df['느낌'][i])

C:\Users\kimhw\AppData\Local\Temp\ipykernel_25584\3503432890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['특징'][i] = ast.literal_eval(df['특징'][i])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_25584\3503432890.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['느낌'][i] = ast.literal_eval(df['느낌'][i])


In [4]:
def graph_maker(df):

    attributes_df, attributes, attributes_merged_df = seperateDF(df, '특징')
    feel_df, feels, feel_merged_df = seperateDF(df,'느낌')

    graph_builder = PandasGraphBuilder()
    graph_builder.add_entities(attributes, '특징_id', '특징')
    graph_builder.add_entities(df, 'ID', '상품')
    graph_builder.add_binary_relations(attributes_merged_df, '특징_id', 'ID', 'define')
    graph_builder.add_binary_relations(attributes_merged_df, 'ID', '특징_id', 'define-by')
    g = graph_builder.build()

    g.nodes['특징'].data['id'] = torch.LongTensor(attributes['특징_id'].cat.codes.values)

    attribute_columns = attributes_df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징','느낌','평점','성분'])
    attributes_df[attribute_columns] = attributes_df[attribute_columns].fillna(False).astype('bool')
    g.nodes['상품'].data['특징'] = torch.FloatTensor(attributes_df[attribute_columns].values)

    attribute_columns = feel_df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징','느낌','평점','성분'])
    feel_df[attribute_columns] = feel_df[attribute_columns].fillna(False).astype('bool')
    g.nodes['상품'].data['느낌'] = torch.FloatTensor(feel_df[attribute_columns].values)

    g.edges['define'].data['rating'] = torch.LongTensor(attributes_merged_df['평점'].values)
    g.edges['define-by'].data['rating'] = torch.LongTensor(attributes_merged_df['평점'].values)

    # 그래프 구조 확인

    print("Number of nodes per type:")
    print(g.number_of_nodes())
    print("Number of edges per type:")
    print(g.number_of_edges())
    print("Node types:")
    print(g.ntypes)
    print("Edge types:")
    print(g.etypes)

    # 노드 데이터 확인
    for ntype in g.ntypes:
        print(f"Node type: {ntype}")
        print(g.nodes[ntype].data)

    # 엣지 데이터 확인
    for stype, etype, dtype in g.canonical_etypes:
        print(f"Edge type: {etype}")
        print(g.edges[etype].data)

    # 그래프 구조 확인
    print("Graph structure:")
    print(g)

    return g

In [5]:
def seperateDF(df, name):
    df_copy = df.copy()

    small_df = df_copy.explode(name)[['ID','상품명',name,'평점']]
    attributes = pd.DataFrame(small_df[name].unique()).reset_index()
    attributes.columns = [name + '_id', name]
    attributes = pd.DataFrame(attributes).astype({name + '_id': 'category'})
    attribute_list = list(attributes[name])
    merged_df = pd.merge(small_df, attributes, on=[name])

    for i in attribute_list:
        data_list = []
        print(i)
        for j in range(len(df)):
            if i in df[name][j]:
                data_list.append(True)
            else:
                data_list.append(False)
        df_copy = pd.concat([df_copy, pd.DataFrame({i: data_list})], axis=1)

    return df_copy, attributes, merged_df

In [6]:
g = graph_maker(df)

output_path = 'dataset/Beauty/beauty_graph.dgl'
dgl.save_graphs(output_path, [g])

dataset = {
    'train-graph': g,
    'context-type': '특징',
    'item-type': '상품',
    'context-to-item-type': 'define',
    'item-to-context-type': 'define-by'}

with open('dataset/Beauty/beauty_data.pickle', 'wb') as f:
    pickle.dump(dataset, f)

선스틱
페이스용
아이용
바디용
모든피부용
SPF50+
PA++++
부드러운 발림
백탁현상방지
촉촉함(수분공급)
저자극
지속력
밀착력
어린이겸용
선스프레이
PA+++
샴푸
일반샴푸
펌프형
세정력
상쾌함
각질케어
청량감
실리콘 프리
시트러스향
아로마향
두피샴푸
비듬케어
모발영양
기타향
모든두피용
쿨샴푸
풍부한 거품
계면활성제 프리
마스크시트
피부진정
피부탄력
영양공급
시트형
에센스
원터치형
패치형
바디클렌저
그린향
향
쿨링감
선파우더/쿠션
50+
피부톤보정
흡수력
리미티드에디션
두피케어
두피토닉
두피진정
탈모샴푸
천연샴푸
비듬샴푸
머릿결개선
유분제거
약산성
손상케어
모든 모발용
파라벤 프리
필오프팩
피지케어
피부결정돈
트러블케어
미스트
파우치형
건성
중건성
복합
지복합
지성
유기자차
노폐물 제거
분사력
레포츠용
워터프루프
무스형
선크림
혼합자차(유기+무기)
50
샴푸바
고체형
모근강화
탈모케어
튜브형
무기자차
티트리
어성초
히알루론산
브라이트닝
리퀴드형
두피스케일러
민감성
PA++
스킨/토너
한국비건인증원
화이트닝
세럼
젤형
오일 프리
볼륨효과
쉐도우형
여드름성완화도움
산성샴푸
뚜껑형
워시오프팩
클렌징비누
넥
무향
팔자주름
이브비건
모발윤기
액상형
탈모증상완화도움
낮
밤
유수분밸런스조절
수분미스트
안개분사
두피에센스
스프레이형
SPF50
모공케어
윤기부여
크림
유향
앰플
피부강화
비건소사이어티
청소년겸용
크림형
안티에이징
주름케어
스틱형
새치케어샴푸
리필형
스포이드형
거품형
핸드케어
피부투명
플로럴향
과일향
콜라겐
임산부겸용
피지조절
가는 모발용
손상 모발용
우디향
미백도움
오일형
주름개선도움
퍼퓸샴푸
피부보호
머스크향
알로에
롤온형
고보습
발림성
주니어겸용
보색샴푸
핸드워시
베이비파우더향
항균
밤형
핸드크림
로션형
퍼스트에센스
시어버터
메이크업베이스 겸용
스팟
스팟케어
남녀공용향수
오 드 퍼퓸
아로마틱
머스크
스프레이
남녀공용
베르가못향
이탈리안 만다린향
암브레트 씨앗향
암브레트향
피어향
플럼향
아이리스향
핑크페퍼향
오 드 코롱
프루티
블랙베리향
월계수 잎향
시

라즈베리샤벳
화이트커런트
프리지아
목련
오렌지블라썸
앰브로
샌슈얼우드
화이트패츌리
아르테미지아향
스파이시어코드향
마테차향
레몬플라워향
산달우드향
엠브로향
센슈얼우드향
버치향
앰버 그리스향
패츄올리 비닐라향
럼향
꼬냑향
인도네시안츌리잎향
통가빈향
버지니아나무향
초록 무화과향
카다몸향
감귤류 과일향
코파이버향
향수세트
쉬어우드향
토르마린향
제이드향
발삼향
사프란어코드향
오렌지블라썸행
디바라벤더 하트향
라반딘하트향
바닐라버본향
허니어코드향
분꽃향
모던머스크향
프루티향
워터리향
베르가모트향
구기자향
히비스커스 플라워향
라일랑향
오리스우드향
오렌지블러썸향
복송아향
래일락향
엠버리노트향
그린 나시향
레드오스만투스향
자스민페달향
그린피치향
자스민페탈향
페촐리향
와일드 리락향
비오렛향
레몬 버바나향
푸제르오리엔탈
프레쉬아로마틱
아이리스
비자리아향
티머페퍼향
플럼블러썸향
핑크 포멜로향
애플 블라썸향
크리미 바닐라향
센티폴리아 로즈향
블랙페허향
골든 앰버향
블랙티바닐라향
아이센스향
캐롯향
스파클링오렌지향
로즈에센셜향
망고향
자몽에센스향
라즈베리넥타향
베르가못미스트향
메론향
프로방스아카시아꽃향
마다가스카버번바닐라향
단델리온향
노란장미향
샤프란향
로즈잎향
머스키향
만다린 잎향
넥멕 토닉향
헤이즐넛 크림향
때죽나무향
로즈 머스크향
그린 시트러스(베르가못)향
라벤더 프로럴(라벤터
화이트 플로럴)향
우디 파우더리(샌들우드
머스크)향
우리 노트향
토바코향
버드추출물향
벨베티오리스향
코튼우드향
치자나무향
솔향
용연향,바닐라향
민트잎향
화이트 타임 앱솔루트향
우롱차잎향
워터 민트향
화이트꽃향
메이트 앱솔루트향
핑크 프리지아향
루밥향
그린 노트향
프레쉬 파인 리브즈향
유칼립투스향
듀이 사이프러스 우드향
소프트 샌들우드향
바이올렛잎사귀향
페탈리아향
그린 헤이즐넛향
로스티드 오크 앱솔루트향
미네랄 어코드향
씨츄세스향
튜버로즈향
만다린오일향
자스민삼박앱솔루트향
블러드 오렌지향
그래니스미스향
물망초향
로사루비기노사향
스티락스향
레드 애플향
핑크 로즈향,자스민향
핑크 로즈향
벨벳향
페티그레인향
대황향


넛메그 에센스향
캐롯씨드 에센스향
시더우드 에센스향
플로랄 노트향
 샌덜우드향
희분꽃향
터키쉬 장미향
장미열매향
바이어로즈향
바이올렛리브즈향
블래커런트향
이탈리안레몬향
체리블로서향
후란지파니향
톤카빈향
화이트 세다우드향
꽃향 
오르쟈 시럽향 
시더우스향
시스투스향
바닐라향 
비터오렌향
페어 윌리엄향
프란지파니향
아쿠아향
스칼렛 포피향
핑크페퍼향,레몬향
쿰쿠아트향 
만디린오렌지향
너트멕향
코리안더 쥬니퍼향
티크향
아로마틱푸제아우디
바이올렛향 잎향
체슈넛향
아카시아 허니향
넥타린 플루트향
호주의 샌달우드향
수수향
파라과이 페티그레인향
나르시스향
가르데니아 로즈향
스위트 오렌지향
부가리안 로즈향
파피러스향
우드 앰버향
피토스포럼향
우디어코드향
루럴 리프향
엠브레트향
우드 세이지향
엠버 우드향
레온 오일향
페어블러썸향
스파클링 베르가못향
레몬시폰향
라이스푸딩어코드향
블루밍자스민향
오카누스향
크랜베리향
로즈힙향
벚꽃나무껍지랴향
망코스틴향
그라스향
초피향
강황향
터키쉬로즈향
암블레트향
그라스 튜베향
아겐우드향
캐달문향
생달우드향
스위트오렌지향
비올라향
베가모트 오일향
블랙큐런트향
뮤거트향
오조닉노트향
슈가레드아몬드어코드향
밀키샌달우드향
크리스탈 앰버향
삼나무 잎향
과이액 나무향
헬리오트로프꽃향
아로마틱향
믹스드 프룻향
로즈 페탈향
시칠리안 만다린
로즈
화이트 헬리오트로프
페츌리
캐롯 에센스향
파촐리에센스향
벤조인 시암 레진향
블랙트러플향
블랙가드니아향
다크초콜릿향
이탈리아오렌지향
진저릴리향
생강뿌리향
화이트티어코드향
화이트삼나무향
실크머스크컴플렉스향
마테앱솔루트향
그레이프풋트향
쿠쿠버향
바이오렛향
엔버향
아니쓰 씨드향
아템시아향
자향
터키쉬장미향
장미 열매향
캐널우드향
재퍼니즈 시더향
자스민 봉우리향
랑군 크리퍼향
과즙향
에델바이스향
바닐라꽃
화이트 머스크
수련어코드향
앱솔루트 민트향
스타 애니스향
레지노이드 인센스향
스타렉스 벤조인향
마다가스카 만다린 오일향
암브레스트향
씨드향
블랙 커런트 꽆봉오리향
오렌지향,그레이프후룻향
소틱후룻향
그린마다린향
베라니움향
새프론향
만

자스만
페치올리
앰브로시드향
샌달우드 머스크향
암브레트시드향
그레이프르풋향
티어코드향
아프리칸 오렌지 플라워향
큐컴버향
엔젤리카향
오렌지 블로썸 앤 비터 오렌지향
쥬니퍼 베리어향
안젤리카 앤 패츌리향
튀니지네롤리향
인도 앱솔루트향
아이리스 앱솔루트향
라오스 벤조인향
세미오리엔탈
글로브향
바닐라향 과닉 우드향
테비버향
바질 그란베르로향
아이스드 애플향
흑장미햐
밀키 머스크향
백샌달우드향
베이 리브즈향
헬르트로프향
바이올렛 페탈향
화이트스웨이드향
다바나에센스향
가이악오일향
인도네시아 패촐리향
바이올린 바니쉬향
호박 송진향
송진향
얼그레이향
코리안더 씨드향
플로랄릴리향
시칠리안향
오렌지블로썸잎향
프티그레인향
머스트향
트랜스페어런트 우드향
코리엔더향
퍼 니들향
포럴향
이모뗄향
오렌지향 디저트향
티히티 티아레향
오포포낵스향
아말피레몬향
파파야플라워향
화이트아마릴리스향
화이트쏜향
레드 로즈향
화이트 조르향
로즈 앱솔루향
만다린 블랙커런트향
로터스플라워향
피어니향
블랙베리 부쉬 리브즈향
릴리 일랑향
센슈얼 우드향
카씨스향
유칼립투스잎향
목향
이탈리안 베르가못 에센스향
오렌지 플라워 페탈향
샴페인 네이쳐 프린트향
패촐리 에센스향
구아익 우드향
블론드 타바코향
아틀라스 시더향
블랙체리
리코라이스향
블랙티잎향
오렌지블로섬향
샌들우드
셀러리씨드향
올스파이스향
마다가스카르산 바닐라향
타로코 오렌지향
락 로즈향
캐롯 씨드향
임모르텔향
너트메그향
구아이약향
케이드향
메이플향
와인 레스향
애플 블러썸향
그린 베르가못향
비가레이드향
네롤리
아프리콧 
샌달우드 
스파클링 오렌지향
로즈 에센셜향
알데하이디향
프레시 감귤향
폴로아향
계피향
톤가향
콩향
비터 그린 오렌지향
코리앤더 씨드향
코리앤더 리브스향
그린 오크모스향
럼 어코드향
이탈리안 레더향
타바코 앱솔루트향
와일드 로즈 어코드향
크리스날 모스향
Juicy Peach
Ripe Pear
Rose Petals
Green Grass
Clary Sage
Lilies
Musk
샹티이 크림향
스파클링 이탈리안 레몬 블랙커런트향
매그놀리아 블러썸향


워터리수국향
하와이안피카케향
블루오키드향
와이드 카멜리아향
핑크 스위이드향
프레쉬 쿨링 민트향
프렌치 프로방스 라벤더향
허벌 믹스향
물쑥향
장미향,향
구르망 노트향
벤자민향
크리스피 페어
아프리카 오렌지꽃 
삼박자스민
바닐라 
캐시미어 우드
플럼향향
로즈향 쟈스민향
프룻
스위트 캔디
솔티향
그레이 엠버향
그레이프 프루프향
Pear
Strawberry
Raspberry
Orange Flower
Peony
Datura
White Musk
안젤리카루트향
삼박자스민앱솔루트향
로지폴리야향
인도네시아파출리향
파우더리어코드향
이모르텔향
이끼류어코드향
스트로베리
싱그러운 피어
목란
시클라맨
트로피컬플로럴향
그레이프프룻
텐저린
씨위드
뱀부
이모뗄 플라워향
시스테 앱솔루트향
호손향
레드만다린향
다임향
브랙페퍼향
릴리오브더밸리
아그루메향
쟈스민티향
팔로산토향
시트런 오일향
화이트 레더 어코드향
만다린 귤향
시칠리안 시트러스향
로즈향 
건과향
락토닉향
연기향
이탈리안 비가라드 오렌지향
브라질리안 오렌지향
파라괴이 안 쁘띠 그향
이지트자스민향
인디안민트향
튀니지오렌지블러썸향
오스트리아 샌달우드향
보라꽃카네이션향
라일락 꽃향
다크베리향
오리스뿌리향
그린카더멈향
터키쉬 로즈페탈향
베지탈 앰버향
미네랄향
모스향,파츌리향
프랑크인센스향
알데하이드 그린향
뮤켓향
엘렐미향
블랙로즈
다바나
마호니엘
징거향
화이트 플로럴 어코드향
만다린 에센스의 싱그러운향
터키쉬 로즈 압솔뤼향
오렌지 블라썸 네롤리의 로맨틱향
레더리 어코드향
안젤리카 씨앗향
바이올렛 어코드향
카디멈
넛멕
큐민
새달우드
그린 코요테향
데저트 페퍼향
아이신스향
텍사스 시더향
워터 머스크향
크레오소트향
일랑일향
화이트플랑워향
앰브레트씨드
휘그향
무스향
로즈다마세나향
아이향
인센스 씨스테 앱솔루트향
인도네시안 패출리향
스타락스향
팔로 산토향
인디안 시프리올향
핑크 장미 꽃잎향
소프트 앰버향
올리브 나무향
빈티지머스크향
브라운 슈가
아쿠아틱 씨 솔트향
로즈 마리향
시트러스 어코드향
스윗 우드향
블랙 페퍼향,
그린 파파야향
푸른 연꽃향
인디

네롤리 플라워 에센스향
카탈록스향
쑥 추출물향
카키향
실리콘 코드향
스파클링 블랙커란트향
튜버로즈 앱솔루트향
베티버 컨센트레이트향
디탁스 우드향
옐로 푸르트향
아르니카향
오스만투스 꽃잎향
얼음향
식물성 호박향
리치장미
장미꽃잎
프리지아벌꿀
벌벳
콰테말라 카다멈향
화이트 머향
올리바넘 레진향
카아라향
피타하야향
만다린 알데하이드향
프란킨센스 레진향
올드 처지 인센스향
발삼 전나무향
우디 스파이스향
체리 어코드향
머암브록산향
린덴블러썸향
라이트 프루트향
플로럴릴리향
세스띠폴리아 로즈향
마린 브리즈향
다마스트 로즈 앱솔루트향
랜티스크 앱솔루트향
알데하이달향
머스크 샌달우드향
암브룩산향
카리스멀향
아가 우드향
아피아시아향
플로럴 오존향
다마세나 로즈향
본디드 와인향
헤디오네향
시더 아틀라스향
앰향
마다가스카르 블랙 바닐라향
머스케논향
블가리안 장미향
파라딘존향
림바놀향
시원한 레몬향
크랜베리바향
아네모네향
산토스산 커피향
버건디카시스향
달콤한 오스만우트향
핑크매그놀리아향
인도네시안 샌달우드 화이트패출리향
츄자향
스위트우드향
아로마 플랜트향
시스트 라다넘향
호주 라벤더향
이탈리 아이리스향
호주 우드향
갈랑가
카더멈
타바코 꽃
주니퍼
은행잎
그레이티크
삼나무
푸르티 노트향
쥬시 노트향
코냑 어코드향
그라스산 레더향
듀베리향
과일노트향
바다향 노트
스위트노트향
베르가모트
주니퍼 베러즈
발삼 피어
머스크,앰버그리스
베러가못향
아모라틱향
프레시향
베티그레인향
스위트앰버향
파우더리 바이올렛향
후레쉬 자스민향
목련꽃잎향
쉬어머스크향
블랙 커런트잎향
센티폴리아 장미 에센스향
그라스 장미향
까씨스향
세갈향
가디니아
티크
호추향
고광나무향
캔디드 오렌지향
카르다몬 오일향
제라늄 앱솔루트향
아이리스 엑스뜨레향
암브레트 버터향
그레이프룻향
로즈 워터향
암브르 씨드향
모헬리 일랑일랑향
차이니즈 목련향
프랑스 수선화 앱솔루트향
배쥬스향
헤이즈향
포메그래네이트향
라즈베리향향
웜향
화이트페출리향
그린 시트러스향
이탈리아산 레몬향
고수씨향
플럼우드향
레몬꽃향
복수아꽃향
인센스 앱솔루트향
화이트머스

망고스틴,오렌지
크랜베리 블러섬
망고스틴
바닐라 오키드
아쿠아틱 코드
불가리안 로즈
모로코산 블루 카모마일
시트러스토닉향
크림자스민향
그라스 튜배로즈향
플라워리 노트
리더 어코드향
해조향
머틀 꽃향
물이끼향
은매화 열매향
다마스세나 로즈향
프리모피오레 레몬향
시프레 어코드향
쿰쾃향
미리틀향
카시스 오일향
코리앤더 씨
앰버 머스크향
블랙 로쿠스트향
넥타리엔향
안젤리카
오렌지 블러썸
파인
당근 꽃향
이탈리안 밀감향
시칠리아풍 베리가못향
생 당근향
시칠리아산 감귤류
콜드 스파이시
우디 노트
프레쉬 프룻
프레쉬 퀸스
와일드 허브 믹스
히솝향
피멘토 베라이트향
히라세움향
스위트 노트
우드 노트
불가리안 라벤더향
화이트 로즈 페탈향
마누카 허니 블러썸 어코드향
아몬드 어코드향
브라질산 커피 추출물향
클레망틴향
타르트 그린 노트향
사과꽃향
넬롤리향
사입러스향
스트로베어리향
선플라워향
꼬리안더향
헬리오트릅향
카스칼론향
헬리오트노프향
옐로우 히비스커스향
인스빈향
커피트향
재스민향 
네츄럴 플로럴향
불가리아 로즈버드향
네츄럴 우디향
엠버 로즈향
오렌지 자스민향
캐러웨이 핑크 페퍼향
카르도맨 앤 바이올렛향
너트맨향
자스민 제라늄향
프랑킨센향
머스크 앤 베티베르향
세이크 어코드향
핑크 프리지아향,오렌지 플라워향,허니 서클향,핑크 로즈향
와일드 자스향
허니 서클향
오리스프루티향
산사나무 꽃향
시너스향
푸룬향
페티 그레인 비가라드향
앱솔루트 튜베로즈향
아쌈향
쿨피향
마스틱 레진향
아프리칸 오렌지플라워향
화이트 가드니아향
레몬그래스향
오스투만스향
프라지파니향
아르태미지아향
만다린트리향
스킨머스향
알데히드 로즈향
휜붗꽃향
뮤것향
그린 프루티향
로즈 다마스향
패추얼리향
프루트 칵테일향
피치아프리콧향
제나리즘향
다마스로즈향
바이브런트 엠버향
머스크 베일향
블랙컬런트향
라떼 어코드향
플로럴햐으 암브록스향
샹틸리향
튜베조르향
로스트통카빈향
스파클링플로럴향
오렌지플라워페탈향
모르코장미향
라벤더햐향
스파클링 페어향
루셔스 로즈 에센셜향
밤부워터향
제주 만다린향
서울매화향
베가모트향,만다린향
샌달우드

Certainty
Rolling Days
To Stay
The Most
Pink Ground
Portion Maker
Common Place
샴페인빔
피치빔
볼륨업
핑크빔
앙코르살사
스터닝키스
본투비피치
196오렌지
베이지크
키치피치
배러댄썬
왓더퍼플
스머지프루프
오토펜슬형
이브닝 레이스
뮬리로브
핑크 튤
샤 로지
쉬어 피치
쉬어 핑크
쉬어 라벤더
누드 블러쉬
무드 블러쉬
드라이 플라워
빈티지 플라워
브러시 내장
다우니 피치
다우니 애프리콧
다우니 코랄
다우니 피그
다우니 핑크
다우니 라벤더
케어링코랄
러빙피치
스마일리샌드
치어리베리
버블리핑크
사과필링
라즈베리무스
살구커스터드
무화과퓨레
크랜베리잼
산딸기 크림인 척
블루베리 머랭인 척
Motion Frame
Creative Filter
Raw Neutrals
호호바씨오일
해바라기씨오일
마카다미아씨오일
01호
02호
데일리카멜
핑크뮬리
데일리애프리콧
피치에이드
로맨틱홀리데이
핑크에이드
이지고잉
라이커블
보니
플로렌스
골드 글로리
코랄 글로리
화이트 글로리
핑크 글로리
모디스트
오나먼트
라르고
피네스
칠링
세레나데
딜라잇
웜 메모리
피치 메모리
뮤즈 모먼트
라일락 모먼트
페이디드 레드
더스티 모브
친애하는 나의 시나몬에게
당신의 최애 애쉬 올림
사랑할 수 밖에 없는 모브로부터
버터카라멜
보타닉가든
포레스트브라운
블루밍가든
영양
호박브라운
Fluffy Red
Simple Stay
Plain
빠져드는달콤함
따뜻한은은함
빛바랜그윽함
탐스러운상큼함
나른한포근함
어스름한 찰나의 오묘함
맴도는 찰나의 아련함
슬림
스키니형
침착보스
다정보스
진지보스
매정보스
예민보스
컬링
포핑
어센틱
헤일로
플레어
소프트 베이지
돌체 로즈
모어 딥 플럼
핑키 모브
폴 인 레드
펄함유
네이키드 샌드 샤인
키스드 글로우
유스 페탈
샤인 블러썸 플레이크
위 속눈썹 전용
아래 속눈썹 전용
쉐이딩 겸용
루시드프리즘
드래곤프리즘
얼티밋프리즘
키튼프리즘
로제프리즘
스텔라프리즘
블라썸프리즘
헤븐스프리즘
피크닉프리즘
서리얼
인스피레이션
엠비셔스
인디케이트


C:\Users\kimhw\AppData\Local\Temp\ipykernel_25584\2015674396.py:13: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:212.)
  g.nodes['특징'].data['id'] = torch.LongTensor(attributes['특징_id'].cat.codes.values)


Number of nodes per type:
128076
Number of edges per type:
1749660
Node types:
['상품', '특징']
Edge types:
['define-by', 'define']
Node type: 상품
{'특징': tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.]]), '느낌': tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.]])}
Node type: 특징
{'id': tensor([   0,    1,    2,  ..., 8537, 8538, 8539])}
Edge type: define-by
{'rating': tensor([4, 4, 4,  ..., 0, 0, 0])}
Edge type: define
{'rating': tensor([4, 4, 4,  ..., 0, 0, 0])}
Graph structure:
Graph(num_nodes={'상품': 119536, '특징': 8540},
      num_edges={('상품', 'define-by', '특징'): 874830, ('특징', 'define', '상품